# 🖼️ Image Classification with BreastMNIST
## A Practical Guide to Medical Image ML (90 minutes)

---

### 📋 Workshop Overview

Welcome! In this tutorial, you'll learn how to build **machine learning models for medical image classification**. We'll work with the BreastMNIST dataset and compare traditional ML methods with deep learning approaches.

### 🎯 What You'll Learn

1. **Data Exploration** - Understanding medical image data
2. **Model Training** - Building and comparing multiple models
3. **Model Evaluation** - Analyzing performance metrics

---

### 📊 About Our Dataset

We're working with **BreastMNIST** - ultrasound images of breast tissue:
- 28x28 grayscale images
- Binary classification: Benign (0) vs Malignant (1)
- Real medical imaging data
- For more information, visit https://medmnist.com/

**Goal**: Classify breast ultrasound images to aid in cancer diagnosis.

---

### ⚙️ Prerequisites Check

First, we need to install the required library.

In [ ]:
!pip install medmnist

Let's verify all required libraries are installed.

In [ ]:
import sys

# Check required libraries and their versions
required_libs = {
    "medmnist": "3.0",
    "numpy": "1.24",
    "scikit-learn": "1.3",
    "matplotlib": "3.7",
    "seaborn": "0.12",
    "tensorflow": "2.13",
    "scipy": "1.10"
}

print("🔍 Checking installed libraries...\n")
print("=" * 70)

for lib_name, min_version in required_libs.items():
    try:
        if lib_name == "scikit-learn":
            import sklearn
            lib = sklearn
            actual_name = "sklearn"
        else:
            lib = __import__(lib_name)
            actual_name = lib_name
        
        installed_version = lib.__version__
        print(f"✓ {lib_name}: {installed_version} (required: >={min_version})")
    except ImportError:
        print(f"✗ {lib_name} is NOT installed. Please install it using:")
        print(f"   pip install {lib_name}>={min_version}")

print("\n✅ Library check complete!")

## 🔍 STEP 1: Data Exploration

Let's understand our medical image dataset before diving into modeling!

### 📥 Loading the Dataset

In [ ]:
from medmnist import BreastMNIST
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

print("📥 Loading BreastMNIST dataset...")
print("=" * 70)

# Load dataset
train_ds = BreastMNIST(split='train', download=True)
val_ds = BreastMNIST(split='val', download=True)
test_ds = BreastMNIST(split='test', download=True)

# Convert to numpy arrays
X_train = train_ds.imgs.reshape(len(train_ds), -1)
y_train = train_ds.labels.flatten()

X_val = val_ds.imgs.reshape(len(val_ds), -1)
y_val = val_ds.labels.flatten()

X_test = test_ds.imgs.reshape(len(test_ds), -1)
y_test = test_ds.labels.flatten()

print(f"\n✅ Dataset loaded successfully!")
print(f"\n📊 Dataset Overview:")
print(f"   Training samples: {len(X_train)}")
print(f"   Validation samples: {len(X_val)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Image dimensions: 28x28 pixels")
print(f"   Flattened features: {X_train.shape[1]} pixels")
print(f"   Classes: {np.unique(y_test)} --> 2 (0=Benign, 1=Malignant)")

### 🔄 Data Preparation for Traditional ML

For traditional ML models (Logistic Regression, Random Forest, etc.), we need to:
1. Flatten images from 28x28 to 784 features
2. Normalize pixel values using StandardScaler

Attention: Make sure to fit the scaler only on the training data to avoid data leakage!

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)  
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

In [ ]:
print("\n🔢 Data Shape After Normalization:")
print("=" * 70)
print(f"   Training set:   {X_train.shape[0]:>5} samples × {X_train.shape[1]:>3} features")
print(f"   Validation set: {X_val.shape[0]:>5} samples × {X_val.shape[1]:>3} features")
print(f"   Test set:       {X_test.shape[0]:>5} samples × {X_test.shape[1]:>3} features")
print(f"\n✅ Data normalized and ready for traditional ML models!")

### 📊 Class Distribution Analysis

Let's check if our dataset is balanced between benign and malignant cases.

In [ ]:
print("\n📊 Analyzing class distribution...")
print("=" * 70)

plt.figure(figsize=(8, 5))

# Separate data by class for individual coloring
benign_data = y_train[y_train == 0]
malignant_data = y_train[y_train == 1]

# Plot each class separately with different colors
plt.hist([benign_data, malignant_data], bins=2, edgecolor="black", 
         color=['#2ecc71', '#e74c3c'], label=['Benign', 'Malignant'])

plt.xticks([0,1], ['Benign', 'Malignant'])
plt.title("Class Distribution in Training Set", fontsize=14, fontweight='bold')
plt.xlabel("Label")
plt.ylabel("Count")
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

benign_count = (y_train==0).sum()
malignant_count = (y_train==1).sum()
total = len(y_train)

print(f"\n📈 Class Distribution:")
print(f"   Benign (0):    {benign_count:>4} samples ({benign_count/total*100:.1f}%)")
print(f"   Malignant (1): {malignant_count:>4} samples ({malignant_count/total*100:.1f}%)")
print(f"   Total:         {total:>4} samples")

if abs(benign_count - malignant_count) / total > 0.2:
    print("\n⚠️  Dataset is imbalanced! Consider using stratified sampling or class weights.")
else:
    print("\n✅ Dataset is reasonably balanced.")

In [ ]:
import random

print("\n🖼️  Sample Images from Training Set:")
print("=" * 70)

# Note: X_train is normalized, we need to use original images
X_train_original = train_ds.imgs

plt.figure(figsize=(12, 6))
for i in range(12):
    idx = random.randint(0, len(X_train_original)-1)
    plt.subplot(2, 6, i+1)
    plt.imshow(X_train_original[idx], cmap="gray")
    label_text = "Benign" if y_train[idx] == 0 else "Malignant"
    plt.title(f"{label_text}", fontsize=9)
    plt.axis("off")
plt.suptitle("Random Sample Images", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 🎨 Dimensionality Visualization with PCA

Let's use PCA to visualize how the two classes separate in 2D space when using raw pixels.

In [ ]:
from sklearn.decomposition import PCA

print("\n🎨 Performing PCA for 2D visualization...")
print("=" * 70)

X_flat = X_train.reshape(len(X_train), -1)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_flat)

print(f"   Explained variance: {pca.explained_variance_ratio_.sum()*100:.2f}%")
print(f"   PC1: {pca.explained_variance_ratio_[0]*100:.2f}%")
print(f"   PC2: {pca.explained_variance_ratio_[1]*100:.2f}%")

plt.figure(figsize=(8, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train, cmap="coolwarm", s=3, alpha=0.6)
plt.title("PCA of BreastMNIST (2D Projection)", fontsize=14, fontweight='bold')
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.colorbar(scatter, label="Label (0=Benign, 1=Malignant)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ PCA visualization complete!")

### 🔍 Image Properties Analysis

**Key Challenge with these Medical Images:** Tumors can appear at different locations in ultrasound images!

❌ **Problem:** Raw pixel values are **location-dependent** - if a tumor appears on the left vs right side, pixel values at specific positions will be completely different.

✅ **Solution approach:** Extract **location-independent features** like texture, contrast, and overall brightness that describe the image regardless of where features appear spatially.

In [ ]:
from scipy.stats import entropy
import numpy as np

print("\n📊 Computing basic image statistics (location-independent)...")
print("=" * 70)

# Use original unnormalized images for meaningful statistics
X_train_original = train_ds.imgs

def compute_image_features(images):
    """Compute location-independent image features"""
    features = {
        'mean_intensity': [],
        'std_intensity': [],
        'contrast': [],
        'entropy': [],
        'brightness': []
    }
    
    for img in images:
        # Overall brightness and variability
        features['mean_intensity'].append(img.mean())
        features['std_intensity'].append(img.std())
        
        # Contrast (range of intensities)
        features['contrast'].append(img.max() - img.min())
        
        # Entropy (texture complexity)
        hist, _ = np.histogram(img.flatten(), bins=50, range=(0, 255))
        hist = hist / hist.sum()
        features['entropy'].append(entropy(hist + 1e-10))  # Add small value to avoid log(0)
        
        # Brightness percentiles
        features['brightness'].append(np.percentile(img, 75))  # 75th percentile
    
    return {k: np.array(v) for k, v in features.items()}

benign_features = compute_image_features(X_train_original[y_train == 0])
malignant_features = compute_image_features(X_train_original[y_train == 1])

print(f"\n📈 Image Feature Statistics:")
print(f"\n   Mean Intensity (overall brightness):")
print(f"      Benign:    {benign_features['mean_intensity'].mean():.2f} ± {benign_features['mean_intensity'].std():.2f}")
print(f"      Malignant: {malignant_features['mean_intensity'].mean():.2f} ± {malignant_features['mean_intensity'].std():.2f}")

print(f"\n   Standard Deviation (local variation):")
print(f"      Benign:    {benign_features['std_intensity'].mean():.2f} ± {benign_features['std_intensity'].std():.2f}")
print(f"      Malignant: {malignant_features['std_intensity'].mean():.2f} ± {malignant_features['std_intensity'].std():.2f}")

print(f"\n   Contrast (max-min intensity range):")
print(f"      Benign:    {benign_features['contrast'].mean():.2f} ± {benign_features['contrast'].std():.2f}")
print(f"      Malignant: {malignant_features['contrast'].mean():.2f} ± {malignant_features['contrast'].std():.2f}")

print(f"\n   Entropy (texture complexity):")
print(f"      Benign:    {benign_features['entropy'].mean():.3f} ± {benign_features['entropy'].std():.3f}")
print(f"      Malignant: {malignant_features['entropy'].mean():.3f} ± {malignant_features['entropy'].std():.3f}")

print("\n💡 These features work regardless of tumor location!")

In [ ]:
import matplotlib.pyplot as plt

print("\n📊 Visualizing feature distributions...")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Contrast comparison
axes[0, 0].hist(benign_features['contrast'], bins=30, alpha=0.6, label="Benign", color='#2ecc71', density=True)
axes[0, 0].hist(malignant_features['contrast'], bins=30, alpha=0.6, label="Malignant", color='#e74c3c', density=True)
axes[0, 0].set_xlabel("Contrast (intensity range)", fontsize=10)
axes[0, 0].set_ylabel("Density", fontsize=10)
axes[0, 0].set_title("Image Contrast Distribution", fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Entropy comparison
axes[0, 1].hist(benign_features['entropy'], bins=30, alpha=0.6, label="Benign", color='#2ecc71', density=True)
axes[0, 1].hist(malignant_features['entropy'], bins=30, alpha=0.6, label="Malignant", color='#e74c3c', density=True)
axes[0, 1].set_xlabel("Entropy (texture complexity)", fontsize=10)
axes[0, 1].set_ylabel("Density", fontsize=10)
axes[0, 1].set_title("Texture Entropy Distribution", fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Std deviation comparison
axes[1, 0].hist(benign_features['std_intensity'], bins=30, alpha=0.6, label="Benign", color='#2ecc71', density=True)
axes[1, 0].hist(malignant_features['std_intensity'], bins=30, alpha=0.6, label="Malignant", color='#e74c3c', density=True)
axes[1, 0].set_xlabel("Standard Deviation", fontsize=10)
axes[1, 0].set_ylabel("Density", fontsize=10)
axes[1, 0].set_title("Pixel Intensity Variability", fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Mean intensity comparison
axes[1, 1].hist(benign_features['mean_intensity'], bins=30, alpha=0.6, label="Benign", color='#2ecc71', density=True)
axes[1, 1].hist(malignant_features['mean_intensity'], bins=30, alpha=0.6, label="Malignant", color='#e74c3c', density=True)
axes[1, 1].set_xlabel("Mean Intensity", fontsize=10)
axes[1, 1].set_ylabel("Density", fontsize=10)
axes[1, 1].set_title("Overall Brightness Distribution", fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Feature visualization complete!")
print("💡 These distributions help identify which features might be useful for classification!")

### 📐 Edge Detection Analysis

Medical images often contain important edge information (tumor boundaries, tissue structures).
Let's analyze edge content using Sobel filters.

In [ ]:
from scipy import ndimage
import numpy as np

print("\n📐 Analyzing edge content using Sobel filters...")
print("=" * 70)

def compute_edge_features(images, n_samples=200):
    """Compute edge strength features"""
    edge_strengths = []
    
    for img in images[:n_samples]:  # Limit for performance
        # Sobel filters for horizontal and vertical edges
        sobel_x = ndimage.sobel(img, axis=0)
        sobel_y = ndimage.sobel(img, axis=1)
        
        # Edge magnitude
        edge_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
        
        # Average edge strength (location-independent)
        edge_strengths.append(edge_magnitude.mean())
    
    return np.array(edge_strengths)

benign_edges = compute_edge_features(X_train_original[y_train == 0])
malignant_edges = compute_edge_features(X_train_original[y_train == 1])

print(f"\n📊 Edge Strength Statistics:")
print(f"   Benign:    {benign_edges.mean():.2f} ± {benign_edges.std():.2f}")
print(f"   Malignant: {malignant_edges.mean():.2f} ± {malignant_edges.std():.2f}")

plt.figure(figsize=(10, 5))
plt.hist(benign_edges, bins=30, alpha=0.6, label="Benign", color='#2ecc71', density=True)
plt.hist(malignant_edges, bins=30, alpha=0.6, label="Malignant", color='#e74c3c', density=True)
plt.xlabel("Average Edge Strength", fontsize=12)
plt.ylabel("Density", fontsize=12)
plt.title("Edge Content Analysis (Sobel Filter)", fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Edge strength measures structural complexity independent of location!")

### 📊 Visualize Sample Images with Edge Detection

Let's see how edge detection highlights structural features.

In [ ]:
print("\n📸 Visualizing sample images with edge detection...")
print("=" * 70)

# Select sample images
benign_idx = np.where(y_train == 0)[0][0]
malignant_idx = np.where(y_train == 1)[0][0]

fig, axes = plt.subplots(2, 3, figsize=(12, 8))

for i, (idx, label) in enumerate([(benign_idx, "Benign"), (malignant_idx, "Malignant")]):
    img = X_train_original[idx]
    
    # Original image
    axes[i, 0].imshow(img, cmap='gray')
    axes[i, 0].set_title(f"{label} - Original", fontweight='bold')
    axes[i, 0].axis('off')
    
    # Sobel X (vertical edges)
    sobel_x = ndimage.sobel(img, axis=0)
    axes[i, 1].imshow(sobel_x, cmap='gray')
    axes[i, 1].set_title("Vertical Edges", fontweight='bold')
    axes[i, 1].axis('off')
    
    # Sobel Y (horizontal edges)
    sobel_y = ndimage.sobel(img, axis=1)
    axes[i, 2].imshow(sobel_y, cmap='gray')
    axes[i, 2].set_title("Horizontal Edges", fontweight='bold')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

print("\n✅ Edge detection visualization complete!")
print("💡 Edges highlight structural boundaries regardless of their location!")

---

## 🔬 CRITICAL INSIGHT: Why Raw Pixels Don't Work Well

Before we start training models, let's understand a fundamental problem:

### ❌ The Problem with Pixel-Based Approaches

When we flatten images to 784 features (28×28 pixels), we're telling the model:
- "Feature 1" = pixel at position (0,0)  
- "Feature 2" = pixel at position (0,1)
- ... and so on

**This creates a huge problem for medical images:**

If a tumor appears at different locations, the "important pixels" change completely! A tumor on the left side has high intensity at different pixel positions than a tumor on the right side.

### ✅ The Solution: Feature Engineering

Instead of using raw pixel positions, we'll extract **location-independent features**:
- **Contrast**: Difference between brightest and darkest areas
- **Entropy**: Texture complexity (regardless of where texture appears)
- **Edge Strength**: How pronounced boundaries are (not where they are)
- **Statistical Moments**: Mean, standard deviation, percentiles

Let's build a feature-based training set and compare it to the pixel-based approach!

### 🛠️ Building a Feature-Based Dataset

Now let's extract meaningful features from all images in our dataset.

In [ ]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage
import pandas as pd

# New:
from skimage.feature import graycomatrix, graycoprops
from skimage.filters import sobel

print("\n🔧 Extracting location-independent features from all images...")
print("=" * 70)

def extract_all_features(images, hist_bins=16, glcm_levels=32):
    """
    Extract a comprehensive set of location-independent features from images.
    These features describe WHAT is in the image, not WHERE it is.
    """
    features_list = []
    
    for img in images:
        # Ensure numpy array & float format
        img = np.array(img).astype(np.float32)
        
        # Normalize to 0–1 if image is in 0–255 range
        if img.max() > 1.0:
            img_norm = img / 255.0
        else:
            img_norm = img.copy()
        
        feature_dict = {}
        
        # ===== Basic Statistics =====
        feature_dict['mean_intensity'] = img.mean()
        feature_dict['std_intensity'] = img.std()
        feature_dict['max_intensity'] = img.max()
        feature_dict['min_intensity'] = img.min()
        feature_dict['median_intensity'] = np.median(img)
        
        # ===== Contrast & Range =====
        feature_dict['contrast'] = img.max() - img.min()
        feature_dict['intensity_range'] = np.ptp(img)  # Peak-to-peak difference
        
        # ===== Distribution Statistics =====
        feature_dict['q25'] = np.percentile(img, 25)
        feature_dict['q75'] = np.percentile(img, 75)
        feature_dict['iqr'] = feature_dict['q75'] - feature_dict['q25']
        feature_dict['skewness'] = ((img - img.mean()) ** 3).mean() / (img.std() ** 3 + 1e-10)
        feature_dict['kurtosis'] = ((img - img.mean()) ** 4).mean() / (img.std() ** 4 + 1e-10)
        
        # ===== Intensity Proportions (Threshold-based) =====
        # Relative fractions of dark / mid / bright pixels (helpful for ultrasound texture)
        feature_dict['frac_low']  = np.mean(img_norm < 0.3)
        feature_dict['frac_mid']  = np.mean((img_norm >= 0.3) & (img_norm <= 0.7))
        feature_dict['frac_high'] = np.mean(img_norm > 0.7)
        
        # ===== Histogram & Entropy (Texture Complexity) =====
        hist, bin_edges = np.histogram(img.flatten(), bins=hist_bins, range=(0, 255))
        hist = hist.astype(np.float32)
        hist = hist / (hist.sum() + 1e-10)
        
        # Histogram bins
        for i in range(hist_bins):
            feature_dict[f'hist_bin_{i}'] = hist[i]
        
        # Shannon entropy
        feature_dict['entropy'] = entropy(hist + 1e-10)
        
        # ===== Edge / Gradient Features (Sobel) =====
        sobel_x = ndimage.sobel(img_norm, axis=0)
        sobel_y = ndimage.sobel(img_norm, axis=1)
        edge_magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
        
        feature_dict['edge_mean'] = edge_magnitude.mean()
        feature_dict['edge_std'] = edge_magnitude.std()
        feature_dict['edge_max'] = edge_magnitude.max()
        feature_dict['edge_energy'] = (edge_magnitude ** 2).mean()
        
        # Fraction of "strong" edges
        thr = np.percentile(edge_magnitude, 90)
        feature_dict['edge_strong_frac'] = np.mean(edge_magnitude > thr)
        
        # ===== Laplacian (Sharpness / Fine Texture) =====
        lap = ndimage.laplace(img_norm)
        feature_dict['laplacian_mean'] = lap.mean()
        feature_dict['laplacian_std'] = lap.std()
        feature_dict['laplacian_energy'] = (lap ** 2).mean()
        
        # ===== GLCM Texture Features (Haralick-style, global) =====
        # Quantize image for GLCM
        img_q = img_norm.copy()
        img_q = np.clip(img_q, 0, 1)
        img_q = (img_q * (glcm_levels - 1)).astype(np.uint8)
        
        distances = [1]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°
        
        glcm = graycomatrix(
            img_q,
            distances=distances,
            angles=angles,
            levels=glcm_levels,
            symmetric=True,
            normed=True
        )
        
        glcm_props = ['contrast', 'energy', 'homogeneity', 'correlation', 'dissimilarity']
        for prop in glcm_props:
            vals = graycoprops(glcm, prop)  # (len(distances), len(angles))
            feature_dict[f'glcm_{prop}_mean'] = vals.mean()
            feature_dict[f'glcm_{prop}_std'] = vals.std()
        
        # ===== Simple Frequency / DCT Features =====
        # 2D Fourier magnitude with light Gaussian smoothing
        dct_x = ndimage.fourier.fourier_gaussian(np.fft.fft2(img_norm), sigma=0.5)
        dct_mag = np.abs(dct_x)
        
        # Low- vs. total frequency energy
        h, w = dct_mag.shape
        center_h, center_w = h // 2, w // 2
        
        # Small centered block = low-frequency area
        low_block = dct_mag[center_h-3:center_h+3, center_w-3:center_w+3]
        feature_dict['dct_low_energy'] = (low_block ** 2).mean()
        feature_dict['dct_total_energy'] = (dct_mag ** 2).mean()
        feature_dict['dct_low_ratio'] = feature_dict['dct_low_energy'] / (feature_dict['dct_total_energy'] + 1e-10)
        
        features_list.append(feature_dict)
    
    return pd.DataFrame(features_list)

# Extract features for all datasets
print("\n   Processing training set...")
X_train_features = extract_all_features(train_ds.imgs)

print("   Processing validation set...")
X_val_features = extract_all_features(val_ds.imgs)

print("   Processing test set...")
X_test_features = extract_all_features(test_ds.imgs)

print(f"\n✅ Feature extraction complete!")
print(f"\n📊 Feature Set Overview:")
print(f"   Number of features: {X_train_features.shape[1]}")
print(f"   Training samples: {X_train_features.shape[0]}")
print(f"   Validation samples: {X_val_features.shape[0]}")
print(f"   Test samples: {X_test_features.shape[0]}")

print(f"\n📋 Extracted Features:")
for i, col in enumerate(X_train_features.columns, 1):
    print(f"   {i:2}. {col}")

print(f"\n📊 Sample Feature Values (first 3 training images):")
display(X_train_features.head(3).round(2))


### 🔄 Normalize Features

Just like with pixel data, we need to normalize our features for optimal model performance.

In [ ]:
from sklearn.preprocessing import StandardScaler

print("\n🔄 Normalizing features...")
print("=" * 70)

# Create and fit scaler on training data
scaler_features = StandardScaler()
X_train_feat_scaled = scaler_features.fit_transform(X_train_features)
X_val_feat_scaled = scaler_features.transform(X_val_features)
X_test_feat_scaled = scaler_features.transform(X_test_features)

print(f"\n✅ Features normalized!")
print(f"\n📊 Comparison:")
print(f"   Pixel-based approach:   {X_train.shape[1]} features (784 pixels)")
print(f"   Feature-based approach: {X_train_feat_scaled.shape[1]} features (handcrafted)")
print(f"\n💡 We reduced dimensionality by {X_train.shape[1] / X_train_feat_scaled.shape[1]:.1f}x while keeping meaningful information!")

## 🤖 STEP 2: Model Training - Comparing Approaches

Now we'll train models using **BOTH approaches** to see which works better:

1. **Pixel-Based**: Raw 784 pixel values (location-dependent)
2. **Feature-Based**: 19 handcrafted features (location-independent)

Let's see which approach wins! 🏆

### 📈 Model 1: Logistic Regression

A simple linear classifier - our baseline model. We'll train it with BOTH approaches.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

print("\n🔄 Training Logistic Regression with BOTH approaches...")
print("=" * 70)

# ===== APPROACH 1: Pixel-Based =====
print("\n1️⃣ Pixel-Based Approach (784 features)")
start = time.time()
logreg_pixels = LogisticRegression(max_iter=200, random_state=42)
logreg_pixels.fit(X_train, y_train)
train_time_pixels = time.time() - start

y_pred_pixels = logreg_pixels.predict(X_test)

logreg_pixels_results = {
    "accuracy":  accuracy_score(y_test, y_pred_pixels),
    "precision": precision_score(y_test, y_pred_pixels),
    "recall":    recall_score(y_test, y_pred_pixels),
    "f1_score":  f1_score(y_test, y_pred_pixels),
    "train_time_sec": train_time_pixels
}

print(f"   ✓ Trained in {train_time_pixels:.2f}s | Accuracy: {logreg_pixels_results['accuracy']:.4f} | F1: {logreg_pixels_results['f1_score']:.4f}")

# ===== APPROACH 2: Feature-Based =====
print("\n2️⃣ Feature-Based Approach (19 features)")
start = time.time()
logreg_features = LogisticRegression(max_iter=200, random_state=42)
logreg_features.fit(X_train_feat_scaled, y_train)
train_time_features = time.time() - start

y_pred_features = logreg_features.predict(X_test_feat_scaled)

logreg_features_results = {
    "accuracy":  accuracy_score(y_test, y_pred_features),
    "precision": precision_score(y_test, y_pred_features),
    "recall":    recall_score(y_test, y_pred_features),
    "f1_score":  f1_score(y_test, y_pred_features),
    "train_time_sec": train_time_features
}

print(f"   ✓ Trained in {train_time_features:.2f}s | Accuracy: {logreg_features_results['accuracy']:.4f} | F1: {logreg_features_results['f1_score']:.4f}")

# ===== COMPARISON =====
print("\n" + "=" * 70)
print("📊 Logistic Regression Comparison")
print("=" * 70)
print(f"{'Metric':<15} {'Pixel-Based':<15} {'Feature-Based':<15} {'Winner':<10}")
print("-" * 70)
print(f"{'Accuracy':<15} {logreg_pixels_results['accuracy']:<15.4f} {logreg_features_results['accuracy']:<15.4f} {'Features' if logreg_features_results['accuracy'] > logreg_pixels_results['accuracy'] else 'Pixels':<10}")
print(f"{'Precision':<15} {logreg_pixels_results['precision']:<15.4f} {logreg_features_results['precision']:<15.4f} {'Features' if logreg_features_results['precision'] > logreg_pixels_results['precision'] else 'Pixels':<10}")
print(f"{'Recall':<15} {logreg_pixels_results['recall']:<15.4f} {logreg_features_results['recall']:<15.4f} {'Features' if logreg_features_results['recall'] > logreg_pixels_results['recall'] else 'Pixels':<10}")
print(f"{'F1-Score':<15} {logreg_pixels_results['f1_score']:<15.4f} {logreg_features_results['f1_score']:<15.4f} {'Features' if logreg_features_results['f1_score'] > logreg_pixels_results['f1_score'] else 'Pixels':<10}")
print(f"{'Training Time':<15} {logreg_pixels_results['train_time_sec']:<15.2f} {logreg_features_results['train_time_sec']:<15.2f} {'Features' if logreg_features_results['train_time_sec'] < logreg_pixels_results['train_time_sec'] else 'Pixels':<10}")

# Store for later comparison
logreg_results = logreg_pixels_results  # Keep pixel-based for backward compatibility

### 🌲 Model 2: Random Forest

An ensemble of decision trees - let's see how it performs with both approaches!

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

print("\n🌲 Training Random Forest with BOTH approaches...")
print("=" * 70)

# ===== APPROACH 1: Pixel-Based =====
print("\n1️⃣ Pixel-Based Approach (784 features)")
start = time.time()
rf_pixels = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_pixels.fit(X_train, y_train)
train_time_pixels = time.time() - start

y_pred_pixels = rf_pixels.predict(X_test)

rf_pixels_results = {
    "accuracy":  accuracy_score(y_test, y_pred_pixels),
    "precision": precision_score(y_test, y_pred_pixels),
    "recall":    recall_score(y_test, y_pred_pixels),
    "f1_score":  f1_score(y_test, y_pred_pixels),
    "train_time_sec": train_time_pixels
}

print(f"   ✓ Trained in {train_time_pixels:.2f}s | Accuracy: {rf_pixels_results['accuracy']:.4f} | F1: {rf_pixels_results['f1_score']:.4f}")

# ===== APPROACH 2: Feature-Based =====
print("\n2️⃣ Feature-Based Approach (19 features)")
start = time.time()
rf_features = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf_features.fit(X_train_feat_scaled, y_train)
train_time_features = time.time() - start

y_pred_features = rf_features.predict(X_test_feat_scaled)

rf_features_results = {
    "accuracy":  accuracy_score(y_test, y_pred_features),
    "precision": precision_score(y_test, y_pred_features),
    "recall":    recall_score(y_test, y_pred_features),
    "f1_score":  f1_score(y_test, y_pred_features),
    "train_time_sec": train_time_features
}

print(f"   ✓ Trained in {train_time_features:.2f}s | Accuracy: {rf_features_results['accuracy']:.4f} | F1: {rf_features_results['f1_score']:.4f}")

# ===== COMPARISON =====
print("\n" + "=" * 70)
print("📊 Random Forest Comparison")
print("=" * 70)
print(f"{'Metric':<15} {'Pixel-Based':<15} {'Feature-Based':<15} {'Winner':<10}")
print("-" * 70)
print(f"{'Accuracy':<15} {rf_pixels_results['accuracy']:<15.4f} {rf_features_results['accuracy']:<15.4f} {'Features' if rf_features_results['accuracy'] > rf_pixels_results['accuracy'] else 'Pixels':<10}")
print(f"{'Precision':<15} {rf_pixels_results['precision']:<15.4f} {rf_features_results['precision']:<15.4f} {'Features' if rf_features_results['precision'] > rf_pixels_results['precision'] else 'Pixels':<10}")
print(f"{'Recall':<15} {rf_pixels_results['recall']:<15.4f} {rf_features_results['recall']:<15.4f} {'Features' if rf_features_results['recall'] > rf_pixels_results['recall'] else 'Pixels':<10}")
print(f"{'F1-Score':<15} {rf_pixels_results['f1_score']:<15.4f} {rf_features_results['f1_score']:<15.4f} {'Features' if rf_features_results['f1_score'] > rf_pixels_results['f1_score'] else 'Pixels':<10}")
print(f"{'Training Time':<15} {rf_pixels_results['train_time_sec']:<15.2f} {rf_features_results['train_time_sec']:<15.2f} {'Features' if rf_features_results['train_time_sec'] < rf_pixels_results['train_time_sec'] else 'Pixels':<10}")

# Store for later comparison
rf_results = rf_pixels_results  # Keep pixel-based for backward compatibility
rf = rf_pixels  # Keep model reference

### 🎯 Model 3: Support Vector Machine (SVM)

A powerful classifier using the RBF kernel - comparing both approaches.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n🎯 Training SVM with BOTH approaches...")
print("=" * 70)

# ===== APPROACH 1: Pixel-Based =====
print("\n1️⃣ Pixel-Based Approach (784 features)")
start = time.time()
svm_pixels = SVC(kernel="rbf", C=5, gamma="scale", random_state=42)
svm_pixels.fit(X_train, y_train)
train_time_pixels = time.time() - start

y_pred_pixels = svm_pixels.predict(X_test)

svm_pixels_results = {
    "accuracy":  accuracy_score(y_test, y_pred_pixels),
    "precision": precision_score(y_test, y_pred_pixels),
    "recall":    recall_score(y_test, y_pred_pixels),
    "f1_score":  f1_score(y_test, y_pred_pixels),
    "train_time_sec": train_time_pixels
}

print(f"   ✓ Trained in {train_time_pixels:.2f}s | Accuracy: {svm_pixels_results['accuracy']:.4f} | F1: {svm_pixels_results['f1_score']:.4f}")

# ===== APPROACH 2: Feature-Based =====
print("\n2️⃣ Feature-Based Approach (19 features)")
start = time.time()
svm_features = SVC(kernel="rbf", C=5, gamma="scale", random_state=42)
svm_features.fit(X_train_feat_scaled, y_train)
train_time_features = time.time() - start

y_pred_features = svm_features.predict(X_test_feat_scaled)

svm_features_results = {
    "accuracy":  accuracy_score(y_test, y_pred_features),
    "precision": precision_score(y_test, y_pred_features),
    "recall":    recall_score(y_test, y_pred_features),
    "f1_score":  f1_score(y_test, y_pred_features),
    "train_time_sec": train_time_features
}

print(f"   ✓ Trained in {train_time_features:.2f}s | Accuracy: {svm_features_results['accuracy']:.4f} | F1: {svm_features_results['f1_score']:.4f}")

# ===== COMPARISON =====
print("\n" + "=" * 70)
print("📊 SVM Comparison")
print("=" * 70)
print(f"{'Metric':<15} {'Pixel-Based':<15} {'Feature-Based':<15} {'Winner':<10}")
print("-" * 70)
print(f"{'Accuracy':<15} {svm_pixels_results['accuracy']:<15.4f} {svm_features_results['accuracy']:<15.4f} {'Features' if svm_features_results['accuracy'] > svm_pixels_results['accuracy'] else 'Pixels':<10}")
print(f"{'Precision':<15} {svm_pixels_results['precision']:<15.4f} {svm_features_results['precision']:<15.4f} {'Features' if svm_features_results['precision'] > svm_pixels_results['precision'] else 'Pixels':<10}")
print(f"{'Recall':<15} {svm_pixels_results['recall']:<15.4f} {svm_features_results['recall']:<15.4f} {'Features' if svm_features_results['recall'] > svm_pixels_results['recall'] else 'Pixels':<10}")
print(f"{'F1-Score':<15} {svm_pixels_results['f1_score']:<15.4f} {svm_features_results['f1_score']:<15.4f} {'Features' if svm_features_results['f1_score'] > svm_pixels_results['f1_score'] else 'Pixels':<10}")
print(f"{'Training Time':<15} {svm_pixels_results['train_time_sec']:<15.2f} {svm_features_results['train_time_sec']:<15.2f} {'Features' if svm_features_results['train_time_sec'] < svm_pixels_results['train_time_sec'] else 'Pixels':<10}")

# Store for later comparison
svm_results = svm_pixels_results  # Keep pixel-based for backward compatibility

### 👥 Model 4: K-Nearest Neighbors (KNN)

Instance-based learning - let's see how it handles location-independent features!

In [ ]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

print("\n🔍 Finding optimal K for KNN (using feature-based approach)...")
print("=" * 70)

k_values = range(1, 51)
accuracies = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_feat_scaled, y_train)
    y_pred = knn.predict(X_test_feat_scaled)
    accuracies.append(accuracy_score(y_test, y_pred))

best_k = k_values[accuracies.index(max(accuracies))]
print(f"\n✅ Best K: {best_k} with accuracy: {max(accuracies):.4f}")

plt.figure(figsize=(10, 5))
plt.plot(k_values, accuracies, marker="o", linewidth=2)
plt.axvline(x=best_k, color='r', linestyle='--', label=f'Best K={best_k}')
plt.xlabel("Number of Neighbors (K)", fontsize=12)
plt.ylabel("Accuracy", fontsize=12)
plt.title("KNN Elbow Criterion - Finding Optimal K", fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

print(f"\n👥 Training KNN with k={best_k} using BOTH approaches...")
print("=" * 70)

# ===== APPROACH 1: Pixel-Based =====
print("\n1️⃣ Pixel-Based Approach (784 features)")
start = time.time()
knn_pixels = KNeighborsClassifier(n_neighbors=best_k)
knn_pixels.fit(X_train, y_train)
train_time_pixels = time.time() - start

y_pred_pixels = knn_pixels.predict(X_test)

knn_pixels_results = {
    "accuracy":  accuracy_score(y_test, y_pred_pixels),
    "precision": precision_score(y_test, y_pred_pixels),
    "recall":    recall_score(y_test, y_pred_pixels),
    "f1_score":  f1_score(y_test, y_pred_pixels),
    "train_time_sec": train_time_pixels
}

print(f"   ✓ Trained in {train_time_pixels:.2f}s | Accuracy: {knn_pixels_results['accuracy']:.4f} | F1: {knn_pixels_results['f1_score']:.4f}")

# ===== APPROACH 2: Feature-Based =====
print("\n2️⃣ Feature-Based Approach (19 features)")
start = time.time()
knn_features = KNeighborsClassifier(n_neighbors=best_k)
knn_features.fit(X_train_feat_scaled, y_train)
train_time_features = time.time() - start

y_pred_features = knn_features.predict(X_test_feat_scaled)

knn_features_results = {
    "accuracy":  accuracy_score(y_test, y_pred_features),
    "precision": precision_score(y_test, y_pred_features),
    "recall":    recall_score(y_test, y_pred_features),
    "f1_score":  f1_score(y_test, y_pred_features),
    "train_time_sec": train_time_features
}

print(f"   ✓ Trained in {train_time_features:.2f}s | Accuracy: {knn_features_results['accuracy']:.4f} | F1: {knn_features_results['f1_score']:.4f}")

# ===== COMPARISON =====
print("\n" + "=" * 70)
print("📊 KNN Comparison")
print("=" * 70)
print(f"{'Metric':<15} {'Pixel-Based':<15} {'Feature-Based':<15} {'Winner':<10}")
print("-" * 70)
print(f"{'Accuracy':<15} {knn_pixels_results['accuracy']:<15.4f} {knn_features_results['accuracy']:<15.4f} {'Features' if knn_features_results['accuracy'] > knn_pixels_results['accuracy'] else 'Pixels':<10}")
print(f"{'Precision':<15} {knn_pixels_results['precision']:<15.4f} {knn_features_results['precision']:<15.4f} {'Features' if knn_features_results['precision'] > knn_pixels_results['precision'] else 'Pixels':<10}")
print(f"{'Recall':<15} {knn_pixels_results['recall']:<15.4f} {knn_features_results['recall']:<15.4f} {'Features' if knn_features_results['recall'] > knn_pixels_results['recall'] else 'Pixels':<10}")
print(f"{'F1-Score':<15} {knn_pixels_results['f1_score']:<15.4f} {knn_features_results['f1_score']:<15.4f} {'Features' if knn_features_results['f1_score'] > knn_pixels_results['f1_score'] else 'Pixels':<10}")
print(f"{'Training Time':<15} {knn_pixels_results['train_time_sec']:<15.2f} {knn_features_results['train_time_sec']:<15.2f} {'Features' if knn_features_results['train_time_sec'] < knn_pixels_results['train_time_sec'] else 'Pixels':<10}")

# Store for later comparison
knn_results = knn_pixels_results  # Keep pixel-based for backward compatibility

### 📊 Model 5: Naive Bayes

A probabilistic classifier - comparing both approaches.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n📊 Training Naive Bayes with BOTH approaches...")
print("=" * 70)

# ===== APPROACH 1: Pixel-Based =====
print("\n1️⃣ Pixel-Based Approach (784 features)")
start = time.time()
nb_pixels = GaussianNB()
nb_pixels.fit(X_train, y_train)
train_time_pixels = time.time() - start

y_pred_pixels = nb_pixels.predict(X_test)

nb_pixels_results = {
    "accuracy":  accuracy_score(y_test, y_pred_pixels),
    "precision": precision_score(y_test, y_pred_pixels),
    "recall":    recall_score(y_test, y_pred_pixels),
    "f1_score":  f1_score(y_test, y_pred_pixels),
    "train_time_sec": train_time_pixels
}

print(f"   ✓ Trained in {train_time_pixels:.2f}s | Accuracy: {nb_pixels_results['accuracy']:.4f} | F1: {nb_pixels_results['f1_score']:.4f}")

# ===== APPROACH 2: Feature-Based =====
print("\n2️⃣ Feature-Based Approach (19 features)")
start = time.time()
nb_features = GaussianNB()
nb_features.fit(X_train_feat_scaled, y_train)
train_time_features = time.time() - start

y_pred_features = nb_features.predict(X_test_feat_scaled)

nb_features_results = {
    "accuracy":  accuracy_score(y_test, y_pred_features),
    "precision": precision_score(y_test, y_pred_features),
    "recall":    recall_score(y_test, y_pred_features),
    "f1_score":  f1_score(y_test, y_pred_features),
    "train_time_sec": train_time_features
}

print(f"   ✓ Trained in {train_time_features:.2f}s | Accuracy: {nb_features_results['accuracy']:.4f} | F1: {nb_features_results['f1_score']:.4f}")

# ===== COMPARISON =====
print("\n" + "=" * 70)
print("📊 Naive Bayes Comparison")
print("=" * 70)
print(f"{'Metric':<15} {'Pixel-Based':<15} {'Feature-Based':<15} {'Winner':<10}")
print("-" * 70)
print(f"{'Accuracy':<15} {nb_pixels_results['accuracy']:<15.4f} {nb_features_results['accuracy']:<15.4f} {'Features' if nb_features_results['accuracy'] > nb_pixels_results['accuracy'] else 'Pixels':<10}")
print(f"{'Precision':<15} {nb_pixels_results['precision']:<15.4f} {nb_features_results['precision']:<15.4f} {'Features' if nb_features_results['precision'] > nb_pixels_results['precision'] else 'Pixels':<10}")
print(f"{'Recall':<15} {nb_pixels_results['recall']:<15.4f} {nb_features_results['recall']:<15.4f} {'Features' if nb_features_results['recall'] > nb_pixels_results['recall'] else 'Pixels':<10}")
print(f"{'F1-Score':<15} {nb_pixels_results['f1_score']:<15.4f} {nb_features_results['f1_score']:<15.4f} {'Features' if nb_features_results['f1_score'] > nb_pixels_results['f1_score'] else 'Pixels':<10}")
print(f"{'Training Time':<15} {nb_pixels_results['train_time_sec']:<15.2f} {nb_features_results['train_time_sec']:<15.2f} {'Features' if nb_features_results['train_time_sec'] < nb_pixels_results['train_time_sec'] else 'Pixels':<10}")

# Store for later comparison
nb_results = nb_pixels_results  # Keep pixel-based for backward compatibility

---

## 📊 Intermediate Summary: Traditional ML Models

Let's create a comprehensive comparison of all traditional ML models we've trained!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

print("\n📊 Comprehensive Model Comparison: Pixel-Based vs Feature-Based")
print("=" * 70)

# Collect all results
comparison_data = {
    'Logistic Regression (Pixels)': logreg_pixels_results,
    'Logistic Regression (Features)': logreg_features_results,
    'Random Forest (Pixels)': rf_pixels_results,
    'Random Forest (Features)': rf_features_results,
    'SVM (Pixels)': svm_pixels_results,
    'SVM (Features)': svm_features_results,
    'KNN (Pixels)': knn_pixels_results,
    'KNN (Features)': knn_features_results,
    'Naive Bayes (Pixels)': nb_pixels_results,
    'Naive Bayes (Features)': nb_features_results,
}

# Create DataFrame
comparison_df = pd.DataFrame(comparison_data).T
comparison_df = comparison_df.round(4)

# Sort by F1-Score
comparison_df_sorted = comparison_df.sort_values('f1_score', ascending=False)

print("\n✅ Results compiled!\n")
display(comparison_df_sorted)

# Calculate average improvement
pixel_models = ['Logistic Regression (Pixels)', 'Random Forest (Pixels)', 'SVM (Pixels)', 'KNN (Pixels)', 'Naive Bayes (Pixels)']
feature_models = ['Logistic Regression (Features)', 'Random Forest (Features)', 'SVM (Features)', 'KNN (Features)', 'Naive Bayes (Features)']

avg_f1_pixels = comparison_df.loc[pixel_models, 'f1_score'].mean()
avg_f1_features = comparison_df.loc[feature_models, 'f1_score'].mean()

print("\n" + "=" * 70)
print("📈 OVERALL COMPARISON")
print("=" * 70)
print(f"Average F1-Score (Pixel-Based):   {avg_f1_pixels:.4f}")
print(f"Average F1-Score (Feature-Based): {avg_f1_features:.4f}")
print(f"Improvement:                       {((avg_f1_features - avg_f1_pixels) / avg_f1_pixels * 100):+.2f}%")
print("=" * 70)

In [ ]:
print("\n📊 Visualizing the comparison...")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: F1-Score comparison
models = ['LogReg', 'RF', 'SVM', 'KNN', 'NB']
pixels_f1 = [logreg_pixels_results['f1_score'], rf_pixels_results['f1_score'], 
             svm_pixels_results['f1_score'], knn_pixels_results['f1_score'], 
             nb_pixels_results['f1_score']]
features_f1 = [logreg_features_results['f1_score'], rf_features_results['f1_score'], 
               svm_features_results['f1_score'], knn_features_results['f1_score'], 
               nb_features_results['f1_score']]

x = range(len(models))
width = 0.35

axes[0].bar([i - width/2 for i in x], pixels_f1, width, label='Pixel-Based', color='#e74c3c', alpha=0.8)
axes[0].bar([i + width/2 for i in x], features_f1, width, label='Feature-Based', color='#2ecc71', alpha=0.8)
axes[0].set_xlabel('Model', fontsize=12)
axes[0].set_ylabel('F1-Score', fontsize=12)
axes[0].set_title('F1-Score Comparison: Pixels vs Features', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(models)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: Training time comparison
pixels_time = [logreg_pixels_results['train_time_sec'], rf_pixels_results['train_time_sec'], 
               svm_pixels_results['train_time_sec'], knn_pixels_results['train_time_sec'], 
               nb_pixels_results['train_time_sec']]
features_time = [logreg_features_results['train_time_sec'], rf_features_results['train_time_sec'], 
                 svm_features_results['train_time_sec'], knn_features_results['train_time_sec'], 
                 nb_features_results['train_time_sec']]

axes[1].bar([i - width/2 for i in x], pixels_time, width, label='Pixel-Based', color='#e74c3c', alpha=0.8)
axes[1].bar([i + width/2 for i in x], features_time, width, label='Feature-Based', color='#2ecc71', alpha=0.8)
axes[1].set_xlabel('Model', fontsize=12)
axes[1].set_ylabel('Training Time (seconds)', fontsize=12)
axes[1].set_title('Training Time Comparison: Pixels vs Features', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(models)
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Visualization complete!")
print("\n💡 Key Takeaway: Feature engineering can significantly improve both performance AND speed!")

### 🔍 Feature Importance Analysis

Let's see which features were most important for our Random Forest model!

In [ ]:
print("\n🔍 Analyzing feature importance from Random Forest (Feature-Based)...")
print("=" * 70)

# Get feature importances
feature_names = X_train_features.columns
importances = rf_features.feature_importances_

# Create DataFrame and sort
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

print("\n📊 Top 10 Most Important Features:")
print("-" * 70)
for idx, row in importance_df.head(10).iterrows():
    print(f"  {row['Feature']:<25} {row['Importance']:.4f}")

# Visualize
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='#2ecc71')
plt.xlabel('Feature Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance in Random Forest (Feature-Based)', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Feature importance analysis complete!")
print("\n💡 This shows which image properties are most discriminative for classification!")
print("   Higher importance = more useful for distinguishing benign vs malignant cases")

---

### 🧠 Deep Learning Approach: Convolutional Neural Network (CNN)

Now let's compare our feature engineering approach with deep learning!

**Key Difference:** CNNs automatically learn features from raw images through convolutional layers, while our traditional ML models needed handcrafted features.

Let's see if the CNN can beat our carefully designed features! 🏆

In [ ]:
import numpy as np
from medmnist import BreastMNIST
from sklearn.model_selection import train_test_split

print("\n🖼️  Preparing data for CNN...")
print("=" * 70)

# Load datasets
train_ds = BreastMNIST(split='train', download=True)
test_ds  = BreastMNIST(split='test',  download=True)

# Images as float32 + normalization to [0,1]
X_full = train_ds.imgs.astype("float32") / 255.0
y_full = train_ds.labels.flatten().astype("int32")

X_test_cnn = test_ds.imgs.astype("float32") / 255.0
y_test_cnn = test_ds.labels.flatten().astype("int32")

# Add channel dimension → (N, 28, 28, 1)
X_full     = X_full[..., np.newaxis]
X_test_cnn = X_test_cnn[..., np.newaxis]

print(f"   Full train shape: {X_full.shape}")
print(f"   Test shape: {X_test_cnn.shape}")

# Create train/validation split
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(
    X_full, y_full,
    test_size=0.2,
    random_state=42,
    stratify=y_full
)

print(f"\n✅ Data prepared for CNN")
print(f"   Training:   {X_train_cnn.shape[0]} samples")
print(f"   Validation: {X_val_cnn.shape[0]} samples")
print(f"   Test:       {X_test_cnn.shape[0]} samples")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC

print("\n🏗️  Building CNN architecture...")
print("=" * 70)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')   # 1 output neuron for binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

print("\n✅ CNN architecture built!")
print("\n📐 Model Architecture:")
model.summary()

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "best_cnn_breastmnist.keras",
    monitor="val_loss",
    save_best_only=True,
    verbose=0
)

print("\n💡 Using Early Stopping (patience=5) and Model Checkpointing")

In [ ]:
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n🚀 Training CNN...")
print("=" * 70)

# Training + Time measurement
start = time.time()

history = model.fit(
    X_train_cnn, y_train_cnn,
    validation_data=(X_val_cnn, y_val_cnn),
    epochs=30,
    batch_size=64,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

train_time = time.time() - start

# Evaluation
y_prob = model.predict(X_test_cnn, verbose=0)
y_pred = (y_prob > 0.5).astype(int).flatten()

cnn_results = {
    "accuracy":        accuracy_score(y_test_cnn, y_pred),
    "precision":       precision_score(y_test_cnn, y_pred),
    "recall":          recall_score(y_test_cnn, y_pred),
    "f1_score":        f1_score(y_test_cnn, y_pred),
    "train_time_sec":  train_time
}

print(f"\n✅ CNN trained in {train_time:.2f} seconds")
print(f"\n📊 Performance Metrics:")
print(f"   Accuracy:  {cnn_results['accuracy']:.4f}")
print(f"   Precision: {cnn_results['precision']:.4f}")
print(f"   Recall:    {cnn_results['recall']:.4f}")
print(f"   F1-Score:  {cnn_results['f1_score']:.4f}")

In [ ]:
import matplotlib.pyplot as plt

print("\n📈 Visualizing training history...")
print("=" * 70)

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Train Acc', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Val Acc', linewidth=2)
plt.title("Accuracy over Epochs", fontsize=12, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Val Loss', linewidth=2)
plt.title("Loss over Epochs", fontsize=12, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 3, 3)
plt.plot(history.history['auc'], label='Train AUC', linewidth=2)
plt.plot(history.history['val_auc'], label='Val AUC', linewidth=2)
plt.title("AUC over Epochs", fontsize=12, fontweight='bold')
plt.xlabel("Epoch")
plt.ylabel("AUC")
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Training visualization complete!")

## 📊 STEP 3: Final Model Comparison & Results

Let's put everything together and see which approach works best!

In [ ]:
import pandas as pd

print("\n📊 Final Comparison: All Approaches")
print("=" * 70)

# Collect best results from each approach
final_results = {
    "LogReg (Pixels)": logreg_pixels_results,
    "LogReg (Features)": logreg_features_results,
    "Random Forest (Pixels)": rf_pixels_results,
    "Random Forest (Features)": rf_features_results,
    "SVM (Pixels)": svm_pixels_results,
    "SVM (Features)": svm_features_results,
    "KNN (Pixels)": knn_pixels_results,
    "KNN (Features)": knn_features_results,
    "Naive Bayes (Pixels)": nb_pixels_results,
    "Naive Bayes (Features)": nb_features_results,
    "CNN (Deep Learning)": cnn_results
}

# Create DataFrame
results_df = pd.DataFrame(final_results).T
results_df = results_df.round(4)

# Sort by F1-Score
results_df = results_df.sort_values('f1_score', ascending=False)

print("\n✅ Results compiled!\n")
print("=" * 70)
print("📈 COMPLETE MODEL COMPARISON TABLE")
print("=" * 70)

display(results_df)

# Find best model
best_model = results_df.index[0]
best_f1 = results_df.loc[best_model, 'f1_score']

print("\n" + "=" * 70)
print(f"🏆 BEST MODEL: {best_model}")
print(f"   F1-Score: {best_f1:.4f}")
print(f"   Accuracy: {results_df.loc[best_model, 'accuracy']:.4f}")
print(f"   Training Time: {results_df.loc[best_model, 'train_time_sec']:.2f} seconds")
print("=" * 70)

# Summary statistics by approach
print("\n📊 Summary by Approach:")
print("-" * 70)

pixel_based = results_df[[('Pixels' in idx or 'CNN' in idx) for idx in results_df.index]]
feature_based = results_df[['Features' in idx for idx in results_df.index]]

print(f"\n  Pixel-Based Models (avg):  F1={pixel_based['f1_score'].mean():.4f}")
print(f"  Feature-Based Models (avg): F1={feature_based['f1_score'].mean():.4f}")
print(f"  CNN (Deep Learning):        F1={cnn_results['f1_score']:.4f}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("\n📊 Visualizing complete model comparison...")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: F1-Score comparison (grouped by model type)
axes[0, 0].barh(range(len(results_df)), results_df['f1_score'], 
                color=['#3498db' if 'Pixels' in idx else '#2ecc71' if 'Features' in idx else '#9b59b6' 
                       for idx in results_df.index])
axes[0, 0].set_yticks(range(len(results_df)))
axes[0, 0].set_yticklabels(results_df.index, fontsize=9)
axes[0, 0].set_xlabel('F1-Score', fontsize=11)
axes[0, 0].set_title('F1-Score Comparison (All Models)', fontsize=12, fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#3498db', label='Pixel-Based'),
    Patch(facecolor='#2ecc71', label='Feature-Based'),
    Patch(facecolor='#9b59b6', label='CNN (Deep Learning)')
]
axes[0, 0].legend(handles=legend_elements, loc='lower right')

# Plot 2: Accuracy comparison
axes[0, 1].barh(range(len(results_df)), results_df['accuracy'], 
                color=['#3498db' if 'Pixels' in idx else '#2ecc71' if 'Features' in idx else '#9b59b6' 
                       for idx in results_df.index])
axes[0, 1].set_yticks(range(len(results_df)))
axes[0, 1].set_yticklabels(results_df.index, fontsize=9)
axes[0, 1].set_xlabel('Accuracy', fontsize=11)
axes[0, 1].set_title('Accuracy Comparison (All Models)', fontsize=12, fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)

# Plot 3: Training time comparison
axes[1, 0].barh(range(len(results_df)), results_df['train_time_sec'], 
                color=['#3498db' if 'Pixels' in idx else '#2ecc71' if 'Features' in idx else '#9b59b6' 
                       for idx in results_df.index])
axes[1, 0].set_yticks(range(len(results_df)))
axes[1, 0].set_yticklabels(results_df.index, fontsize=9)
axes[1, 0].set_xlabel('Training Time (seconds)', fontsize=11)
axes[1, 0].set_title('Training Time Comparison', fontsize=12, fontweight='bold')
axes[1, 0].grid(axis='x', alpha=0.3)

# Plot 4: Precision vs Recall scatter
for idx in results_df.index:
    if 'Pixels' in idx:
        color = '#3498db'
    elif 'Features' in idx:
        color = '#2ecc71'
    else:
        color = '#9b59b6'
    
    axes[1, 1].scatter(results_df.loc[idx, 'recall'], 
                      results_df.loc[idx, 'precision'], 
                      s=150, alpha=0.7, color=color)
    
axes[1, 1].set_xlabel('Recall', fontsize=11)
axes[1, 1].set_ylabel('Precision', fontsize=11)
axes[1, 1].set_title('Precision vs Recall Trade-off', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)
axes[1, 1].legend(handles=legend_elements)

plt.tight_layout()
plt.show()

print("\n✅ Visualization complete!")

---

## 🎓 Summary & Key Takeaways

### 📝 What We Learned

#### 1. **The Location Problem in Medical Imaging**
   - Raw pixel values are **location-dependent** - tumors at different positions create completely different pixel patterns
   - This makes learning difficult for traditional ML models

#### 2. **Feature Engineering Solution**
   - We extracted **19 location-independent features** (contrast, entropy, edge strength, etc.)
   - These features describe WHAT is in the image, not WHERE it is
   - Result: Often **better performance** with **41x fewer features** (19 vs 784)

#### 3. **Approach Comparison**

| Approach | # Features | Pros | Cons |
|----------|-----------|------|------|
| **Pixel-Based** | 784 | Simple, no preprocessing | Location-dependent, high dimensionality |
| **Feature-Based** | 19 | Location-independent, interpretable, fast | Requires domain knowledge |
| **CNN (Deep Learning)** | Learned | Automatic feature learning, powerful | Needs more data, slower training, black box |

### 🔑 Key Insights

#### Performance Insights:
- **Feature engineering matters!** Handcrafted features often matched or beat pixel-based approaches
- **CNNs excel at image tasks** because they automatically learn spatial features through convolutions
- **Trade-offs exist**: Traditional ML with good features trains faster; CNNs potentially perform better with more data

#### Medical Context:
- **High recall is critical** in cancer detection - we want to catch all malignant cases
- **Interpretable features** (contrast, texture) help clinicians understand the model's reasoning
- **Fast inference** matters in clinical settings - feature-based models predict instantly

### 💡 Practical Lessons

1. **Start with feature engineering** - understand your problem domain
2. **Compare approaches** - don't assume deep learning is always best
3. **Consider constraints** - training time, interpretability, data availability
4. **Medical AI requires** - high reliability, interpretability, and validation

### 🚀 Next Steps

To further improve:
- **Hyperparameter tuning** (GridSearchCV) for all models
- **More sophisticated features** (texture patterns, shape descriptors)
- **Data augmentation** for CNN (rotation, flipping, brightness adjustment)
- **Ensemble methods** combining multiple models
- **Transfer learning** using pre-trained medical imaging networks
- **Cross-validation** for more robust performance estimates

---

**🎉 Congratulations! You've completed a comprehensive medical image classification workshop!**

**Key Message:** Feature engineering isn't dead - it's a powerful tool that can match or complement deep learning, especially when you understand your domain and have limited data!